# Road Segmentation Project


In [1]:
# Imports
import math
import os
import re
import cv2
import torch
import numpy as np
import parameters as params
import utils
import trainer
from processing import augment
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp
from glob import glob
from random import sample
from PIL import Image
from torch import nn
from train import train
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from utils.datasets import ImageDataset
#from utils.dataset import ImageDataset, load_all_data
from utils.losses import DiceBCELoss
from utils import utils 

In [2]:
# Loading data
device = 'cuda' if torch.cuda.is_available() else 'cpu'
images_org = utils.load_images(os.path.join(params.ROOT_PATH, 'training', 'images'), False)
masks_org = utils.load_images(os.path.join(params.ROOT_PATH, 'training', 'groundtruth'), True)

In [3]:
## Uptraining
import sys
import wandb
from pathlib import Path
from tqdm import tqdm

CHECKPOINT_PATH = Path("checkpoints")
DATA_PATH = Path("data")


In [4]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 1)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

    
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=3, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=3, shuffle=True)
    
model = smp.Unet(
    encoder_name="vgg19",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model = model.to(device)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,
'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
#scheduler = ReduceLROnPlateau(optimizer)
train(model, optimizer, train_dataloader, val_dataloader, loss_fn, 40, None, 0, metric_fns)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: bgueney (besteguney). Use `wandb login --relogin` to force relogin


Epoch 1/40 Validation: 100%|██████████| 5/5 [00:00<00:00, 17.65it/s, val_loss=0.56] 


	- loss = 0.6499349121437517
  	- val_loss = 0.5598637342453003
  	- acc = 0.1928956199350745
  	- val_acc = 0.2765001118183136
  	- f1_score = 0.41601959182772524
  	- val_f1_score = 0.5780626654624939
 


Epoch 2/40 Validation: 100%|██████████| 5/5 [00:00<00:00, 21.26it/s, val_loss=0.474]


	- loss = 0.5461209146089332
  	- val_loss = 0.47441973686218264
  	- acc = 0.41038217932678933
  	- val_acc = 0.6683534979820251
  	- f1_score = 0.6080390779778014
  	- val_f1_score = 0.7136200189590454
 


...

Epoch 39/40 Validation: 100%|██████████| 5/5 [00:00<00:00, 13.29it/s, val_loss=0.197]


	- loss = 0.05443415669507758
  	- val_loss = 0.19719380140304565
  	- acc = 0.981456832830296
  	- val_acc = 0.9323346376419067
  	- f1_score = 0.9476720108542331
  	- val_f1_score = 0.8041820406913758
 


Epoch 40/40 Validation: 100%|██████████| 5/5 [00:00<00:00, 20.74it/s, val_loss=0.189]


	- loss = 0.053924201532851816
  	- val_loss = 0.18930282592773437
  	- acc = 0.9818069844744927
  	- val_acc = 0.933419692516327
  	- f1_score = 0.9480974986109623
  	- val_f1_score = 0.8120941519737244
 


{0: {'loss': 0.6499349121437517,
  'val_loss': 0.5598637342453003,
  'acc': 0.1928956199350745,
  'val_acc': 0.2765001118183136,
  'f1_score': 0.41601959182772524,
  'val_f1_score': 0.5780626654624939},
 1: {'loss': 0.5461209146089332,
  'val_loss': 0.47441973686218264,
  'acc': 0.41038217932678933,
  'val_acc': 0.6683534979820251,
  'f1_score': 0.6080390779778014,
  'val_f1_score': 0.7136200189590454},
 2: {'loss': 0.4914928279643835,
  'val_loss': 0.4620865821838379,
  'acc': 0.6831225330053374,
  'val_acc': 0.6844699501991272,
  'f1_score': 0.6692769880904708,
  'val_f1_score': 0.7055099368095398},
 3: {'loss': 0.4357698823130408,
  'val_loss': 0.39168444871902464,
  'acc': 0.7762826112813728,
  'val_acc': 0.8128449559211731,
  'f1_score': 0.7140458070261534,
  'val_f1_score': 0.752538549900055},
 4: {'loss': 0.37148461508196456,
  'val_loss': 0.36177008152008056,
  'acc': 0.857266403214876,
  'val_acc': 0.8550677299499512,
  'f1_score': 0.7504369983839434,
  'val_f1_score': 0.74372

In [5]:
model_backs = []
model_backs.append(model)

In [6]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 1)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

    
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=3, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=3, shuffle=True)
    
model = smp.Unet(
    encoder_name="efficientnet-b4",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model = model.to(device)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,
'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
#scheduler = ReduceLROnPlateau(optimizer)
train(model, optimizer, train_dataloader, val_dataloader, loss_fn, 40, None, 0, metric_fns)

acc,▁▃▅▆▇▇▇█████████████████████████████████
f1_score,▁▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇█████████████████████
loss,█▇▆▅▅▄▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▅▅▇▇███████████████▇███████████████████
val_f1_score,▁▅▅▆▆▇▇▇█▇▇██▇██████▆██▇████████████████
val_loss,█▆▆▅▄▃▃▂▂▂▂▁▁▂▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,0.98181
f1_score,0.9481
loss,0.05392
val_acc,0.93342
val_f1_score,0.81209


Epoch 1/40 Validation: 100%|██████████| 5/5 [00:00<00:00, 20.34it/s, val_loss=0.578]


	- loss = 0.6299140508784804
  	- val_loss = 0.5776807427406311
  	- acc = 0.4212282986141915
  	- val_acc = 0.4159803628921509
  	- f1_score = 0.44902481398610183
  	- val_f1_score = 0.4779277503490448
 


Epoch 2/40 Validation: 100%|██████████| 5/5 [00:00<00:00, 20.88it/s, val_loss=0.448]


	- loss = 0.4582691767881083
  	- val_loss = 0.448132848739624
  	- acc = 0.7664944266164025
  	- val_acc = 0.726689088344574
  	- f1_score = 0.6398790000483047
  	- val_f1_score = 0.6112017869949341
 


...

Epoch 39/40 Validation: 100%|██████████| 5/5 [00:00<00:00, 21.06it/s, val_loss=0.198]


	- loss = 0.08432532257811967
  	- val_loss = 0.19826749563217164
  	- acc = 0.9710673724496087
  	- val_acc = 0.9282140731811523
  	- f1_score = 0.9173839272454728
  	- val_f1_score = 0.8037587165832519
 


Epoch 40/40 Validation: 100%|██████████| 5/5 [00:00<00:00, 20.51it/s, val_loss=0.196]


	- loss = 0.08327106403750043
  	- val_loss = 0.19599637985229493
  	- acc = 0.9717368656812713
  	- val_acc = 0.9291802287101746
  	- f1_score = 0.9183181926261547
  	- val_f1_score = 0.8058395028114319
 


{0: {'loss': 0.6299140508784804,
  'val_loss': 0.5776807427406311,
  'acc': 0.4212282986141915,
  'val_acc': 0.4159803628921509,
  'f1_score': 0.44902481398610183,
  'val_f1_score': 0.4779277503490448},
 1: {'loss': 0.4582691767881083,
  'val_loss': 0.448132848739624,
  'acc': 0.7664944266164025,
  'val_acc': 0.726689088344574,
  'f1_score': 0.6398790000483047,
  'val_f1_score': 0.6112017869949341},
 2: {'loss': 0.375850658084071,
  'val_loss': 0.35616310834884646,
  'acc': 0.8521170276542043,
  'val_acc': 0.8239949584007263,
  'f1_score': 0.6997198847144149,
  'val_f1_score': 0.6986963391304016},
 3: {'loss': 0.31776328419530114,
  'val_loss': 0.2934208631515503,
  'acc': 0.8861315382081408,
  'val_acc': 0.8926626801490783,
  'f1_score': 0.7406432233577551,
  'val_f1_score': 0.753587543964386},
 4: {'loss': 0.27855242754137793,
  'val_loss': 0.26383707523345945,
  'acc': 0.9039075929065084,
  'val_acc': 0.9059484481811524,
  'f1_score': 0.7656888303368591,
  'val_f1_score': 0.77159839

In [7]:
model_backs.append(model)

In [8]:
model4 = smp.Unet(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model4 = model4.to(device)

checkpoint = torch.load('checkpoints/lively-surf-85/epoch_40.pt')
model4.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [9]:
model_backs.append(model4)

In [10]:
models = []
for model_b in model_backs:
    train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

    images_aug, masks_aug = augment.augment_data(train_images, train_masks, 2)
    
    images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
    masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)
    
    val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
    val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)
    
    # reshape the image to simplify the handling of skip connections and maxpooling
    train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
    val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))
    
        
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=3, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=3, shuffle=True)
        
    loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
    metric_fns = {'acc': trainer.accuracy_fn,
    'f1_score': trainer.f1_score_fn}
    optimizer = torch.optim.Adam(model_b.parameters(), lr=1e-5)
    #scheduler = ReduceLROnPlateau(optimizer)
    train(model_b, optimizer, train_dataloader, val_dataloader, loss_fn, 15, None, 0, metric_fns)
    models.append(model_b)

acc,▁▅▆▇▇▇▇▇████████████████████████████████
f1_score,▁▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████████████████
loss,█▆▅▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▅▇▇████████████████████████████████████
val_f1_score,▁▄▆▇▇█▇▇▇███████████████████████████████
val_loss,█▆▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,0.97174
f1_score,0.91832
loss,0.08327
val_acc,0.92918
val_f1_score,0.80584


Epoch 1/15 Validation: 100%|██████████| 5/5 [00:00<00:00, 22.27it/s, val_loss=0.181]


	- loss = 0.15414152450339738
  	- val_loss = 0.18103957176208496
  	- acc = 0.9473700186078863
  	- val_acc = 0.9356160640716553
  	- f1_score = 0.8473773289096448
  	- val_f1_score = 0.820695436000824
 


Epoch 2/15 Validation: 100%|██████████| 5/5 [00:00<00:00, 21.53it/s, val_loss=0.187]


	- loss = 0.140454442464104
  	- val_loss = 0.18684723377227783
  	- acc = 0.9519195672153502
  	- val_acc = 0.9350540637969971
  	- f1_score = 0.8610599073328713
  	- val_f1_score = 0.8148595452308655
 


...

	- loss = 0.09572081695231356
  	- val_loss = 0.1951848030090332
  	- acc = 0.9672885088957557
  	- val_acc = 0.9290147662162781
  	- f1_score = 0.9055876916693163
  	- val_f1_score = 0.8060663104057312
 


Epoch 14/15 Validation: 100%|██████████| 5/5 [00:00<00:00, 26.73it/s, val_loss=0.195]


	- loss = 0.09220765912255575
  	- val_loss = 0.19454236030578614
  	- acc = 0.9680398226708404
  	- val_acc = 0.9296563029289245
  	- f1_score = 0.909054697484009
  	- val_f1_score = 0.8067293643951416
 


Epoch 15/15 Validation: 100%|██████████| 5/5 [00:00<00:00, 29.45it/s, val_loss=0.196]


	- loss = 0.0901086270347122
  	- val_loss = 0.1955119013786316
  	- acc = 0.9688183346460032
  	- val_acc = 0.9304104447364807
  	- f1_score = 0.9111074089079865
  	- val_f1_score = 0.805574107170105
 


In [13]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 1)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

    
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=3, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=3, shuffle=True)
    
model = smp.Unet(
    encoder_name="efficientnet-b5",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model = model.to(device)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,
'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
#scheduler = ReduceLROnPlateau(optimizer)
train(model, optimizer, train_dataloader, val_dataloader, loss_fn, 40, None, 0, metric_fns)
model_backs.append(model)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b5-b6417697.pth" to /home/bgueney/.cache/torch/hub/checkpoints/efficientnet-b5-b6417697.pth
100%|██████████| 117M/117M [00:00<00:00, 234MB/s]  


acc,▁▂▃▄▅▅▆▆▆▇▇▇▇██
f1_score,▁▃▃▄▅▅▆▆▆▇▇▇▇██
loss,█▆▆▅▄▄▃▃▃▂▂▂▂▁▁
val_acc,▃█▆▇▇▅▆▃▅▆▄▄▁▃▅
val_f1_score,▅▇▇▅▂█▄▆▁▇▄▃▃▃▂
val_loss,▄▂▂▄▇▁▅▃█▁▅▆▆▆▆
acc,0.96882
f1_score,0.91111
loss,0.09011
val_acc,0.93041
val_f1_score,0.80557


Epoch 1/40 Validation: 100%|██████████| 5/5 [00:00<00:00, 14.60it/s, val_loss=0.634]


	- loss = 0.639579470074454
  	- val_loss = 0.6344097256660461
  	- acc = 0.4413619880066362
  	- val_acc = 0.23751039505004884
  	- f1_score = 0.4261844920990772
  	- val_f1_score = 0.38510274291038515
 


Epoch 2/40 Validation: 100%|██████████| 5/5 [00:00<00:00, 12.37it/s, val_loss=0.473]


...

	- loss = 0.08069510515346083
  	- val_loss = 0.19589039087295532
  	- acc = 0.9723133091316667
  	- val_acc = 0.9300021767616272
  	- f1_score = 0.9211523810098338
  	- val_f1_score = 0.8054389119148254
 


Epoch 39/40 Validation: 100%|██████████| 5/5 [00:00<00:00, 16.11it/s, val_loss=0.203]


	- loss = 0.08021542014077652
  	- val_loss = 0.20250754356384276
  	- acc = 0.9724932077319123
  	- val_acc = 0.9304940700531006
  	- f1_score = 0.9215279089850049
  	- val_f1_score = 0.7986833214759826
 


Epoch 40/40 Validation: 100%|██████████| 5/5 [00:00<00:00, 15.84it/s, val_loss=0.2]  


	- loss = 0.07970453278962956
  	- val_loss = 0.19997992515563964
  	- acc = 0.9727806164774784
  	- val_acc = 0.9289966702461243
  	- f1_score = 0.92197010406228
  	- val_f1_score = 0.8014433145523071
 


In [15]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 1)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

    
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=3, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=3, shuffle=True)
    
model = smp.Unet(
    encoder_name="resnext50_32x4d",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model = model.to(device)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,
'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
#scheduler = ReduceLROnPlateau(optimizer)
train(model, optimizer, train_dataloader, val_dataloader, loss_fn, 40, None, 0, metric_fns)
model_backs.append(model)

Epoch 1/40 Validation: 100%|██████████| 5/5 [00:00<00:00, 17.26it/s, val_loss=0.514]


	- loss = 0.6232163178366285
  	- val_loss = 0.5135270953178406
  	- acc = 0.44188283730384914
  	- val_acc = 0.7211579561233521
  	- f1_score = 0.452154844826044
  	- val_f1_score = 0.5887404799461364
 


Epoch 2/40 Validation: 100%|██████████| 5/5 [00:00<00:00, 16.47it/s, val_loss=0.44] 


	- loss = 0.4890970525353454
  	- val_loss = 0.43988810777664183
  	- acc = 0.7435898940230525
  	- val_acc = 0.7746817350387574
  	- f1_score = 0.6030351838400198
  	- val_f1_score = 0.6500019073486328
 


Epoch 3/40 Validation: 100%|██████████| 5/5 [00:00<00:00, 17.23it/s, val_loss=0.372]


...

	- loss = 0.07314331725586293
  	- val_loss = 0.19912219047546387
  	- acc = 0.9753286360308181
  	- val_acc = 0.9304583668708801
  	- f1_score = 0.9287995321806087
  	- val_f1_score = 0.8022729277610778
 


Epoch 39/40 Validation: 100%|██████████| 5/5 [00:00<00:00, 17.36it/s, val_loss=0.194]


	- loss = 0.0687742649122726
  	- val_loss = 0.19429396390914916
  	- acc = 0.9764038733271665
  	- val_acc = 0.9302110552787781
  	- f1_score = 0.932926744222641
  	- val_f1_score = 0.8072084188461304
 


Epoch 40/40 Validation: 100%|██████████| 5/5 [00:00<00:00, 15.28it/s, val_loss=0.203]


	- loss = 0.06898277720739675
  	- val_loss = 0.20325053930282594
  	- acc = 0.976489434408587
  	- val_acc = 0.928682017326355
  	- f1_score = 0.9327499908070231
  	- val_f1_score = 0.7980822801589966
 


In [16]:
for model_b in model_backs[-2:]:
    train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

    images_aug, masks_aug = augment.augment_data(train_images, train_masks, 2)
    
    images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
    masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)
    
    val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
    val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)
    
    # reshape the image to simplify the handling of skip connections and maxpooling
    train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
    val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))
    
        
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=3, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=3, shuffle=True)
        
    loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
    metric_fns = {'acc': trainer.accuracy_fn,
    'f1_score': trainer.f1_score_fn}
    optimizer = torch.optim.Adam(model_b.parameters(), lr=1e-5)
    #scheduler = ReduceLROnPlateau(optimizer)
    train(model_b, optimizer, train_dataloader, val_dataloader, loss_fn, 15, None, 0, metric_fns)
    models.append(model_b)

acc,▁▅▆▇▇▇▇▇▇███████████████████████████████
f1_score,▁▃▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇███████████████████
loss,█▆▅▅▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▃▆▆▆▇▇▇▇███████████████████████████████
val_f1_score,▁▃▅▅▆▆▇▇▇▇▇▇▇▇▇▇█▇██▇▇█▇▇██▇█████▇█▇▇███
val_loss,█▆▅▄▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,0.97649
f1_score,0.93275
loss,0.06898
val_acc,0.92868
val_f1_score,0.79808


Epoch 1/15 Validation: 100%|██████████| 5/5 [00:00<00:00, 15.69it/s, val_loss=0.194]


	- loss = 0.16988279791765434
  	- val_loss = 0.19444148540496825
  	- acc = 0.9410663450411124
  	- val_acc = 0.9291341304779053
  	- f1_score = 0.8313640642535779
  	- val_f1_score = 0.8070642590522766
 


Epoch 2/15 Validation: 100%|██████████| 5/5 [00:00<00:00, 14.09it/s, val_loss=0.194]


	- loss = 0.16740213022675626
  	- val_loss = 0.19366695880889892
  	- acc = 0.943245811055797
  	- val_acc = 0.9299131155014038
  	- f1_score = 0.8337680304697318
  	- val_f1_score = 0.8078105330467225
 


...

	- loss = 0.09201058445050735
  	- val_loss = 0.20067498683929444
  	- acc = 0.9684674892314645
  	- val_acc = 0.9303828477859497
  	- f1_score = 0.909173588882121
  	- val_f1_score = 0.8008015751838684
 


Epoch 14/15 Validation: 100%|██████████| 5/5 [00:00<00:00, 16.96it/s, val_loss=0.196]


	- loss = 0.08942192608071853
  	- val_loss = 0.19596179723739623
  	- acc = 0.9695234668347262
  	- val_acc = 0.9306577563285827
  	- f1_score = 0.9117350467415744
  	- val_f1_score = 0.8056256413459778
 


Epoch 15/15 Validation: 100%|██████████| 5/5 [00:00<00:00, 16.54it/s, val_loss=0.193]


	- loss = 0.08737908269083777
  	- val_loss = 0.1929005742073059
  	- acc = 0.970093707705653
  	- val_acc = 0.9306283473968506
  	- f1_score = 0.9137768491294033
  	- val_f1_score = 0.8085163950920105
 


In [19]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 1)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

    
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=3, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=3, shuffle=True)
    
model = smp.Unet(
    encoder_name="vgg19",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model = model.to(device)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,
'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
#scheduler = ReduceLROnPlateau(optimizer)
train(model, optimizer, train_dataloader, val_dataloader, loss_fn, 40, None, 0, metric_fns)

acc,▁▂▃▄▅▅▆▆▆▇▇▇███
f1_score,▁▂▃▄▅▅▆▆▆▇▇▇███
loss,█▇▆▅▄▄▃▃▃▂▂▂▁▁▁
val_acc,▅▄█▆█▄▅▁█▇▇▇▇█▇
val_f1_score,▆▆▇▃▆▅█▂▇▇▃▂▁▅▇
val_loss,▃▃▂▆▃▄▁▇▂▂▆▇█▄▂
acc,0.97009
f1_score,0.91378
loss,0.08738
val_acc,0.93063
val_f1_score,0.80852


Epoch 1/40 Validation: 100%|██████████| 5/5 [00:00<00:00, 19.54it/s, val_loss=0.453]


	- loss = 0.5975693124671315
  	- val_loss = 0.45328282117843627
  	- acc = 0.5037622867628585
  	- val_acc = 0.7963727116584778
  	- f1_score = 0.4674729502998119
  	- val_f1_score = 0.6777179718017579
 


Epoch 2/40 Validation: 100%|██████████| 5/5 [00:00<00:00, 20.90it/s, val_loss=0.374]


	- loss = 0.43759031212607097
  	- val_loss = 0.37424545288085936
  	- acc = 0.8041376395280971
  	- val_acc = 0.8376279473304749
  	- f1_score = 0.6425883790781332
  	- val_f1_score = 0.6966404795646668
 


...

Epoch 39/40 Validation: 100%|██████████| 5/5 [00:00<00:00, 21.47it/s, val_loss=0.198]


	- loss = 0.06176572414331658
  	- val_loss = 0.19794124364852905
  	- acc = 0.979144673014796
  	- val_acc = 0.930644178390503
  	- f1_score = 0.93984910221987
  	- val_f1_score = 0.8035508751869201
 


Epoch 40/40 Validation: 100%|██████████| 5/5 [00:00<00:00, 21.48it/s, val_loss=0.195]


	- loss = 0.06066068521765775
  	- val_loss = 0.1945432662963867
  	- acc = 0.9791689868583235
  	- val_acc = 0.931929087638855
  	- f1_score = 0.9408157024272653
  	- val_f1_score = 0.8064455032348633
 


{0: {'loss': 0.5975693124671315,
  'val_loss': 0.45328282117843627,
  'acc': 0.5037622867628585,
  'val_acc': 0.7963727116584778,
  'f1_score': 0.4674729502998119,
  'val_f1_score': 0.6777179718017579},
 1: {'loss': 0.43759031212607097,
  'val_loss': 0.37424545288085936,
  'acc': 0.8041376395280971,
  'val_acc': 0.8376279473304749,
  'f1_score': 0.6425883790781332,
  'val_f1_score': 0.6966404795646668},
 2: {'loss': 0.35982942303945853,
  'val_loss': 0.3167565822601318,
  'acc': 0.8706666418286257,
  'val_acc': 0.8978131294250489,
  'f1_score': 0.7026318983976231,
  'val_f1_score': 0.7351886868476868},
 3: {'loss': 0.31541128075400066,
  'val_loss': 0.28064876794815063,
  'acc': 0.8914816545885663,
  'val_acc': 0.8957139730453492,
  'f1_score': 0.7305446627528168,
  'val_f1_score': 0.7586414098739624},
 4: {'loss': 0.27844842506009476,
  'val_loss': 0.25283768177032473,
  'acc': 0.9080908915331197,
  'val_acc': 0.9210092186927795,
  'f1_score': 0.7562799772550893,
  'val_f1_score': 0.7

In [ ]:
model4 = smp.Unet(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model4 = model4.to(device)

checkpoint = torch.load('checkpoints/lively-surf-85/epoch_40.pt')
model4.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
#scheduler = ReduceLROnPlateau(optimizer)
train2(model4, optimizer, images_org, masks_org, loss_fn, 15, None, 0, metric_fns)

In [ ]:
model1 = smp.Unet(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model1 = model1.to(device)
model2 = smp.Unet(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model2 = model2.to(device)

model3 = smp.Unet(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model3 = model3.to(device)

In [ ]:
checkpoint = torch.load('checkpoints/divine-thunder-64/epoch_15.pt')
model1.load_state_dict(checkpoint['model_state_dict'])

checkpoint = torch.load('checkpoints/worldly-snow-83/epoch_15.pt')
model2.load_state_dict(checkpoint['model_state_dict'])

checkpoint = torch.load('checkpoints/dutiful-donkey-84/epoch_15.pt')
model3.load_state_dict(checkpoint['model_state_dict'])



In [ ]:
utils.create_submission("test", "images",'worldly_snow.csv', model2, device)

In [12]:
len(models)

3

In [18]:
# Making prediction
test_path = os.path.join(params.ROOT_PATH, "test", "images")
test_filenames = (glob(test_path + '/*.png'))
test_images = utils.load_all_from_path(test_path)
batch_size = test_images.shape[0]
size = test_images.shape[1:3]

test_images = np.stack([cv2.resize(img, dsize=(384, 384)) for img in test_images], 0)
test_images = test_images[:, :, :, :3]
test_images = utils.np_to_tensor(np.moveaxis(test_images, -1, 1), device)

preds = utils.ensemble_predict(models, [1,1,1,1,1], test_images)

test_pred = np.stack([cv2.resize(img, dsize=size) for img in preds], 0)  # resize to original shape
# now compute labels
test_pred = test_pred.reshape((-1, size[0] // params.PATCH_SIZE, params.PATCH_SIZE, size[0] // params.PATCH_SIZE, params.PATCH_SIZE))
test_pred = np.moveaxis(test_pred, 2, 3)
test_pred = np.round(np.mean(test_pred, (-1, -2)) > params.CUTOFF)
with open("ensemble_staged_training_5_diff.csv", 'w') as f:
        f.write('id,prediction\n')
        for fn, patch_array in zip(sorted(test_filenames), test_pred):
            img_number = int(re.search(r"satimage_(\d+)", fn).group(1))
            for i in range(patch_array.shape[0]):
                for j in range(patch_array.shape[1]):
                    f.write("{:03d}_{}_{},{}\n".format(img_number, j*params.PATCH_SIZE, i*params.PATCH_SIZE, int(patch_array[i, j])))
                    

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 1)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=2, shuffle=True)
    
model2 = smp.Unet(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model2 = model2.to(device)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,
'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.Adam(model2.parameters(), lr=1e-4)
#scheduler = ReduceLROnPlateau(optimizer)
train(model2, optimizer, train_dataloader, val_dataloader, loss_fn, 20, None, 0, metric_fns)

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 2)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=2, shuffle=True)
    
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,
'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.Adam(model2.parameters(), lr=1e-5)
#scheduler = ReduceLROnPlateau(optimizer)
train(model2, optimizer, train_dataloader, val_dataloader, loss_fn, 15, None, 0, metric_fns)

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 3)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=2, shuffle=True)
    
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,
'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.Adam(model2.parameters(), lr=1e-5)
#scheduler = ReduceLROnPlateau(optimizer)
train(model2, optimizer, train_dataloader, val_dataloader, loss_fn, 15, None, 0, metric_fns)

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 1)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

    
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=3, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=3, shuffle=True)
    
model = smp.Unet(
    encoder_name="resnet101",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model = model.to(device)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,
'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
#scheduler = ReduceLROnPlateau(optimizer)
train(model, optimizer, train_dataloader, val_dataloader, loss_fn, 20, None, 0, metric_fns)

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 2)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

    
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=3, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=3, shuffle=True)

loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,
'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
#scheduler = ReduceLROnPlateau(optimizer)
train(model, optimizer, train_dataloader, val_dataloader, loss_fn, 20, None, 0, metric_fns)

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 1)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

    
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=3, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=3, shuffle=True)
    
model = smp.Unet(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model = model.to(device)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,
'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
#scheduler = ReduceLROnPlateau(optimizer)
train(model, optimizer, train_dataloader, val_dataloader, loss_fn, 40, None, 0, metric_fns)

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 2)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

    
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=2, shuffle=True)
    
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,
'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
#scheduler = ReduceLROnPlateau(optimizer)
train(model, optimizer, train_dataloader, val_dataloader, loss_fn, 15, None, 0, metric_fns)

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 2)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=2, shuffle=True)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
#scheduler = ReduceLROnPlateau(optimizer)
train(model, optimizer, train_dataloader, val_dataloader, loss_fn, 20, None, 0, metric_fns)

In [ ]:
import ttach as tta

transforms = tta.Compose(
    [
        tta.HorizontalFlip(),
        tta.VerticalFlip(),
        tta.Rotate90([0,90])
    ]
)

tta_models = []
for i in range(5):
    tta_models.append(tta.SegmentationTTAWrapper(models[i], transforms))

preds = utils.ensemble_predict(tta_models, [1,1,1,1,1], test_images)
test_pred = np.stack([cv2.resize(img, dsize=size) for img in preds], 0)  # resize to original shape
# now compute labels
test_pred = test_pred.reshape((-1, size[0] // params.PATCH_SIZE, params.PATCH_SIZE, size[0] // params.PATCH_SIZE, params.PATCH_SIZE))
test_pred = np.moveaxis(test_pred, 2, 3)
test_pred = np.round(np.mean(test_pred, (-1, -2)) > params.CUTOFF)
with open("ensemble_resunet_50_5_aug_tta.csv", 'w') as f:
        f.write('id,prediction\n')
        for fn, patch_array in zip(sorted(test_filenames), test_pred):
            img_number = int(re.search(r"satimage_(\d+)", fn).group(1))
            for i in range(patch_array.shape[0]):
                for j in range(patch_array.shape[1]):
                    f.write("{:03d}_{}_{},{}\n".format(img_number, j*params.PATCH_SIZE, i*params.PATCH_SIZE, int(patch_array[i, j])))

In [ ]:
models.append(model)

In [ ]:
models.append(model)

In [ ]:
utils.create_submission("test", "images",'resnet_trained_further.csv', model, device)

In [ ]:
from torch.optim.lr_scheduler import CosineAnnealingLR

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
)

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 2)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

    
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=2, shuffle=True)
    

model = models[0].to(device)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
scheduler = CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6)
train(model, optimizer, train_dataloader, val_dataloader, loss_fn, 20, scheduler, 0, metric_fns)

In [ ]:
utils.create_submission("test", "images",'resnet_trained_even_further.csv', model, device)

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
)

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 2)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

    
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=2, shuffle=True)
    

model = models[1].to(device)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
scheduler = ReduceLROnPlateau(optimizer)
train(model, optimizer, train_dataloader, val_dataloader, loss_fn, 15, scheduler, 0, metric_fns)